In [11]:
import os, sys
os.chdir("/home/agnes/Skyclean")       # repo root
sys.path.insert(0, os.getcwd())

from skyclean.silc.ilc import ProduceSILC
from skyclean.silc.file_templates import FileTemplates
from skyclean.silc.pipeline import Pipeline

In [12]:
import sys, jax
print(sys.executable)      # should be .../miniconda3/envs/clean_cmb/bin/python
print(jax.__version__)     # 0.5.x
print(jax.devices())       # [CudaDevice(...)]

/home/agnes/miniconda3/envs/clean_cmb/bin/python
0.5.3
[CudaDevice(id=0), CudaDevice(id=1)]


In [13]:
import numpy as np

# --- Physical constants ---
h = 6.62607015e-34        # Planck [J·s]
k = 1.380649e-23          # Boltzmann [J/K]
T_cmb = 2.7255            # [K]

# --- Planck frequency channels in GHz ---
frequencies = ['030','044','070','100','143','217','353','545','857']
nu = np.array([float(f) for f in frequencies]) * 1e9   # Hz
x = h * nu / (k * T_cmb)

# Unit conversion (antenna→thermodynamic), proportional to dB/dT
g_nu = (x**2 * np.exp(x)) / (np.exp(x) - 1.0)**2

# (a) CMB in thermodynamic units: flat response
cmb_vec = np.ones_like(nu)

# (b) tSZ in thermodynamic units
tsz_vec = x * ((np.exp(x) + 1) / (np.exp(x) - 1)) - 4
# Optional: tsz_vec is ~0 near 217 GHz automatically; don't zero 30/44/70.

# (c) Synchrotron: antenna T ∝ ν^β, convert to thermodynamic
beta_s = -3.1
nu0 = 30e9
T_sync_ant = (nu / nu0) ** beta_s
sync_vec = T_sync_ant / g_nu

# Normalize columns (helps matching)
def _norm(v): 
    n = np.linalg.norm(v)
    return v if n == 0 else (v / n)

reference_vectors = {
    "cmb":  _norm(cmb_vec),
    "tsz":  _norm(tsz_vec),
    "sync": _norm(sync_vec),
}

# Column order in F (controls what “column index” each name maps to)
F_cols = ["cmb", "tsz", "sync"]
F = np.column_stack([reference_vectors[name] for name in F_cols])  # shape (Nfreq, Nc)

def build_f(F, component_name):
    """Return unit vector f (Nc,) with 1 at the column corresponding to component_name."""
    # allow list/tuple to mean multiple constraints; here: single name
    if isinstance(component_name, (list, tuple)):
        raise ValueError("build_f expects a single name; pass one of: " + ", ".join(F_cols))
    try:
        j = F_cols.index(component_name.lower())
    except ValueError:
        raise ValueError(f"Component '{component_name}' not in F columns {F_cols}")
    f = np.zeros(F.shape[1], dtype=float)
    f[j] = 1.0
    return f


In [14]:
print("F columns:", F_cols)
print("f for cmb:", build_f(F, "cmb"))
print("f for tsz:", build_f(F, "tsz"))
print("f for sync:", build_f(F, "sync"))

F columns: ['cmb', 'tsz', 'sync']
f for cmb: [1. 0. 0.]
f for tsz: [0. 1. 0.]
f for sync: [0. 0. 1.]


In [15]:
directory = "/Scratch/agnes/data" 
ft = FileTemplates(directory).file_templates

# INPUT wavelet coeff template (uses {comp}, {frequency}, {scale}, {realisation:05d}, {lmax}, {lam})
file_template = ft.get("wavelet_coeffs") or ft["wavelet_c_j"]

# OUTPUT templates expected by ILC_wav_coeff_maps_MP (use {component}, {extract_comp}, {realisation})
output_templates = {
    "doubled_maps":           ft["doubled_maps"],
    "covariance_matrices":    ft["covariance_matrices"],
    "weight_vector_matrices": ft["weight_vector_matrices"],
    "ilc_maps":               ft["ilc_maps"],
    "trimmed_maps":           ft["trimmed_maps"],
    "ilc_synth":              ft["ilc_synth"],
    "ilc_spectrum":           ft.get("ilc_spectrum"),  # optional
}


In [16]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

pipe = Pipeline(
    components=["cmb","sync","noise","tsz"],
    wavelet_components=["cfn"],
    ilc_components=["cmb"],
    frequencies=["030","044","070","100","143","217","353","545","857"],
    realisations=1,
    start_realisation=0,
    lmax=64,
    N_directions=1,
    lam=2.0,
    method="jax_cuda",
    visualise=True,
    save_ilc_intermediates=False,
    overwrite=False,
    directory="/Scratch/agnes/data",
)

In [17]:
pipe.step_download()

--- STARTING DATA DOWNLOAD ---
File /Scratch/agnes/data/CMB_realisations/sync_f030.fits already exists. Skipping download.
File /Scratch/agnes/data/CMB_realisations/sync_f044.fits already exists. Skipping download.
File /Scratch/agnes/data/CMB_realisations/sync_f070.fits already exists. Skipping download.
File /Scratch/agnes/data/CMB_realisations/sync_f100.fits already exists. Skipping download.
File /Scratch/agnes/data/CMB_realisations/sync_f143.fits already exists. Skipping download.
File /Scratch/agnes/data/CMB_realisations/sync_f217.fits already exists. Skipping download.
File /Scratch/agnes/data/CMB_realisations/sync_f353.fits already exists. Skipping download.
File /Scratch/agnes/data/CMB_realisations/sync_f545.fits already exists. Skipping download.
File /Scratch/agnes/data/CMB_realisations/sync_f857.fits already exists. Skipping download.
File /Scratch/agnes/data/CMB_realisations/tsz_f030.fits already exists. Skipping download.
File /Scratch/agnes/data/CMB_realisations/tsz_f044

In [18]:
pipe.step_process()

--- PROCESSING CFNs AND TOTAL MAP CFN ---
CFN map at 030 GHz for realisation 0 already exists. Skipping processing.
CFN map at 044 GHz for realisation 0 already exists. Skipping processing.
CFN map at 070 GHz for realisation 0 already exists. Skipping processing.
CFN map at 100 GHz for realisation 0 already exists. Skipping processing.
CFN map at 143 GHz for realisation 0 already exists. Skipping processing.
CFN map at 217 GHz for realisation 0 already exists. Skipping processing.
CFN map at 353 GHz for realisation 0 already exists. Skipping processing.
CFN map at 545 GHz for realisation 0 already exists. Skipping processing.
CFN map at 857 GHz for realisation 0 already exists. Skipping processing.


In [19]:
pipe.step_wavelets()

--- PRODUCING WAVELET TRANSFORMS ---
/Scratch/agnes/data/wavelet_transforms/wavelet_coeffs/cfn_wavelet_f030_s0_r00000_lmax64_lam2.0.npy
Wavelet coefficients for cfn at 030 GHz r0 exist. Skipping.
Wavelet transform for cfn at 030 GHz for realisation 0 saved.
/Scratch/agnes/data/wavelet_transforms/wavelet_coeffs/cfn_wavelet_f044_s0_r00000_lmax64_lam2.0.npy
Wavelet coefficients for cfn at 044 GHz r0 exist. Skipping.
Wavelet transform for cfn at 044 GHz for realisation 0 saved.
/Scratch/agnes/data/wavelet_transforms/wavelet_coeffs/cfn_wavelet_f070_s0_r00000_lmax64_lam2.0.npy
Wavelet coefficients for cfn at 070 GHz r0 exist. Skipping.
Wavelet transform for cfn at 070 GHz for realisation 0 saved.
/Scratch/agnes/data/wavelet_transforms/wavelet_coeffs/cfn_wavelet_f100_s0_r00000_lmax64_lam2.0.npy
Wavelet coefficients for cfn at 100 GHz r0 exist. Skipping.
Wavelet transform for cfn at 100 GHz for realisation 0 saved.
/Scratch/agnes/data/wavelet_transforms/wavelet_coeffs/cfn_wavelet_f143_s0_r0000

In [20]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   # use GPU with physical id=1

target_components = "cmb"
scales = [0, 1, 2, 3, 4, 5, 6, 7, 8]
realisations = ["0000"]

for realisation in realisations:
    print(f"Processing realisation {realisation} for targets: {target_components}")

    constrained_maps = ProduceSILC.ILC_wav_coeff_maps_MP(
        file_template=file_template,
        frequencies=frequencies,         # strings like "030","044",...
        scales=scales,
        realisations=[realisation],       # ints; input template uses :05d internally
        output_templates=output_templates,
        L_max=64,
        N_directions=1,
        comp="cfn",                      # INPUT mixture tag (matches {comp} in wavelet template)
        constraint=True,
        F=F,
        extract_comp=target_components,   # target name used in filenames & constraint
        reference_vectors=reference_vectors,
    )

Processing realisation 0000 for targets: cmb
Processing realisation 0000 for component cfn


2025-08-25 15:30:22.553701: W external/xla/xla/stream_executor/cuda/cuda_command_buffer.cc:668] Retry CUDA graph instantiation after OOM error
E0825 15:30:22.553890 1388480 pjrt_stream_executor_client.cc:3077] Execution of replica 0 failed: RESOURCE_EXHAUSTED: Underlying backend ran out of memory trying to instantiate graph with 12 nodes and 0 conditionals (total of 0 alive graphs in the process). You can try to (a) Give more memory to the driver by reducing XLA_CLIENT_MEM_FRACTION (b) Disable command buffers with 'XLA_FLAGS=--xla_gpu_enable_command_buffer=' (empty set). Original error: Failed to instantiate CUDA graph: CUDA_ERROR_OUT_OF_MEMORY: out of memory


XlaRuntimeError: RESOURCE_EXHAUSTED: Underlying backend ran out of memory trying to instantiate graph with 12 nodes and 0 conditionals (total of 0 alive graphs in the process). You can try to (a) Give more memory to the driver by reducing XLA_CLIENT_MEM_FRACTION (b) Disable command buffers with 'XLA_FLAGS=--xla_gpu_enable_command_buffer=' (empty set). Original error: Failed to instantiate CUDA graph: CUDA_ERROR_OUT_OF_MEMORY: out of memory

In [22]:
import os
# pick GPU 1
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# be memory-friendly (preallocation off + smaller fraction + disable CUDA command buffers)
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.6"
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"] = "platform"
os.environ["XLA_FLAGS"] = "--xla_gpu_enable_command_buffer=false"


In [23]:
import jax
print(jax.devices())  # should show a single GpuDevice(id=0) — that's your physical GPU 1


[CudaDevice(id=0), CudaDevice(id=1)]
